# Speeding up an Alignment Algorithm. 

## TinyDB ftw!

---

### An example of an alignment:

### Does the pattern "cac" occur in the following sequence?

### cactaagcacacagagaata

### yes!

### *cac* taag *cacac* agagaata

---

### Here is a fast algroithm to determine if a pattern is found in a reference.

In [1]:
def find_in(reference, pattern):
    if pattern in reference:
        return "Yes"

In [2]:
reference = 'cactaagcacacagagaataatgtctagaatctgagtgccatgttatcaaattgtactgagactcttgcagtcacacaggct'
pattern = 'cac'

In [3]:
find_in(reference, pattern)

'Yes'

### Great! But this does not give us location information or allow us to perform fuzzy alignments. 

### This function iterates through the referene using a 'window' the same size as the pattern.

In [43]:
def pattern_find(reference, pattern):
    pattern_location = []  # Creates empty list to store location of alignments. 
    for i, base in enumerate(reference):
        query_pattern = reference[i:i+len(pattern)]  # "Window" used for search.
        if query_pattern == pattern:
            pattern_location.append(i) 
    return(pattern_location)

In [5]:
pattern_find(reference, pattern)

[0, 7, 9, 72, 74]

### This is slow when there are lots (hundreds) of patterns to be quered because it has to iterate step-wise through the reference for each pattern.

In [65]:
pattern_list = ['acac','cag','ttat','agaa','ttatcaaatt', 'aaaa']

In [7]:
for pattern in pattern_list:
    print('{} {}'.format(pattern, pattern_find(reference, pattern)))

acac [8, 73]
cag [11, 68, 76]
ttat [43]
agaa [14, 26]
ttatcaaatt [43]


In [8]:
from tinydb import TinyDB, Query
import re
import json

In [9]:
db = TinyDB('/Users/ksindy/PycharmProjects/oligo_search_website/pratice_db.json')

In [62]:
def pattern_find_tinydb(pattern_list, reference):
    
    aligns = []  # Creates empty list to store alignments
    pattern_length_list = []  # Creates empty list to store pattern lengths
    
    for pattern in pattern_list:  
        if len(pattern) not in pattern_length_list:  
            pattern_length_list.append(len(pattern))
            
        for i, nucleotide in enumerate(reference):  
            ref_chunk = reference[i:i+len(pattern)]  
            db.insert({'sequence': ref_chunk, 'index':i+1})  
                
        sequence_match = Query()
        matches = db.search(sequence_match.sequence == pattern)
        matches = json.dumps(matches)
        locations = re.findall(r'\d+', matches)
        aligns.append('{} {}'.format(pattern, locations))
        
    return(aligns)

In [66]:
pattern_find_tinydb(pattern_list, reference)

["acac ['9', '74']",
 "cag ['12', '69', '77']",
 "ttat ['44', '44']",
 "agaa ['15', '27', '15', '27', '15', '27']",
 "ttatcaaatt ['44']",
 'aaaa []']

In [67]:
db.purge()

In [105]:
def pattern_find_tinydb(pattern_list, reference):
    
    aligns = []  # Creates empty list to store alignments
    pattern_length_list = []  # Creates empty list to store pattern lengths

    for pattern in pattern_list:  
        
        '''
        DETERMINE LENGTH OF PATTERN
        Do not need to add to db if pattern length already seen
        '''
        
        if len(pattern) not in pattern_length_list:  
            pattern_length_list.append(len(pattern))
            
        '''
        ADD REFERENCE TO DATABASE
        Splits reference into pattern length chunks and adds to the db 
        along with location (i).
        [{'sequence': 'cact', 'index': 1}]
        [{'sequence': 'cact', 'index': 1}, {'sequence': 'acta', 'index': 2}]
        ...
        [{'sequence': 'cac', 'index': 1}]
        [{'sequence': 'cac', 'index': 1}, {'sequence': 'act', 'index': 2}]

        '''

            for i, nucleotide in enumerate(reference):  
                ref_chunk = reference[i:i+len(pattern)]  
                db.insert({'sequence': ref_chunk, 'index':i+1}) 

            '''
            FIND ALIGNMENTS
            [{"sequence": "acac", "index": 9}, {"sequence": "acac", "index": 74}]
            [{"sequence": "cag", "index": 12}, {"sequence": "cag", "index": 69}...
            [{"sequence": "ttat", "index": 44}]
            [{"sequence": "agaa", "index": 15}, {"sequence": "agaa", "index": 27}]
            [{"sequence": "ttatcaaatt", "index": 44}]
            '''
        sequence_match = Query()
        matches = db.search(sequence_match.sequence == pattern)  # bit-like object
        matches = json.dumps(matches)  # Dumps json to python dict, in this case to string
        locations = re.findall(r'\d+', matches)
        print(matches)
        aligns.append('{} {}'.format(pattern, locations))
    return(aligns)

IndentationError: unexpected indent (<ipython-input-105-55f715084728>, line 28)

In [366]:
final_list = []
working_list = []

def neighbors(pattern, mismatch_num):
    nucleotides = 'ACGT'
    current_list = []
    final_list = [] # this will mess up the recursive algorithm if more than 1 mismatch

#     for pattern in list:
    pattern_whole = ''
    final_list.append(pattern)
    for index, nucleotide in enumerate(pattern):
        rest = pattern[(index+1)::]

        for i, nuc in enumerate(nucleotides):
            working_pattern = ''
            working_pattern += pattern_whole
            working_pattern += nuc
            working_pattern += rest

            if working_pattern not in final_list:
                final_list.append(working_pattern)
                current_list.append(working_pattern)
        pattern_whole += nucleotide

    if mismatch_num > 1:
        for item in current_list:
            working_list.append(item)
        return neighbors(working_list, mismatch_num-1)
    else:
        #return 'this is result{}'.format("\n".join(final_list))
        return final_list
final_list = []
working_list = []

In [367]:
print(neighbors('CACT', 1))

['CACT', 'AACT', 'GACT', 'TACT', 'CCCT', 'CGCT', 'CTCT', 'CAAT', 'CAGT', 'CATT', 'CACA', 'CACC', 'CACG']


In [371]:
def pattern_find_tinydb(pattern_list, reference, mismatch_num):
    
    aligns = []  # Creates empty list to store alignments
    pattern_length_list = []  # Creates empty list to store pattern lengths
    reference_upper = reference.upper().replace(' ','')
    reference_clean = re.sub('[^A-Z]+', '', reference_upper)
    
    for pattern in pattern_list:  
        string_upper = pattern.upper().replace(' ','')  # Removes whitespace, converts to caps.
        string_clean = re.sub('[^A-Z]+', '', string_upper)  # Uses a regex to remove char not a A-Z.
        #print(string_clean)
        
        if len(pattern) not in pattern_length_list:  
            pattern_length_list.append(len(pattern))
            
            for i, nucleotide in enumerate(reference_clean):  
                ref_chunk = reference_clean[i:i+len(pattern)]  
                db.insert({'sequence': ref_chunk, 'index':i+1})  
        aligns.append(pattern)     
        all_matches = ''
        for neighbor in neighbors(string_clean, mismatch_num):
            matches = db.search(Query().sequence == neighbor)
            matches = json.dumps(matches)    
            all_matches += matches
        locations = re.findall(r'\d+', all_matches)
        list1 = [int(x) for x in locations]
        list1.sort()
        aligns.append(list1)
    return(aligns)

In [372]:
db.purge()
final_list = []
working_list = []


In [373]:
print(pattern_find_tinydb(pattern_list, reference, 1))
#cactaagcacacagagaataatgtctagaatctgagtgccatgttatcaaattgtactgagactcttgcagtcacacaggct

['acac', [7, 9, 11, 60, 62, 72, 74, 76], 'cag', [1, 5, 8, 10, 12, 14, 26, 32, 34, 40, 48, 57, 59, 69, 73, 75, 77], 'ttat', [19, 44, 52], 'agaa', [6, 13, 15, 18, 27, 60], 'ttatcaaatt', [44], 'aaaa', [15, 17, 18, 27, 48, 49]]


In [317]:
def mismatch(string1, string2):
    mismatches = 0
    for (nucleotide1, nucleotide2) in zip(string1, string2):
        if nucleotide1 != nucleotide2:
            mismatches += 1
    return(mismatches)

In [374]:
def approximate_patterns(text, pattern_list, max_mismatches):
    reference_upper = reference.upper().replace(' ','')
    reference_clean = re.sub('[^A-Z]+', '', reference_upper)
    
    pattern_matches = []
    
    for pattern in pattern_list:
        pattern_matches.append(pattern)
        for i, base in enumerate(text):
            query_pattern = text[i:i+len(pattern)]
            if mismatch(pattern, query_pattern) <= max_mismatches:
                pattern_matches.append(i+1)
    return(pattern_matches)
#print(approximate_patterns(text, pattern, max_mismatch))


In [375]:
print(approximate_patterns(reference, pattern_list, 1))

['acac', 7, 9, 11, 60, 62, 72, 74, 76, 82, 'cag', 1, 5, 8, 10, 12, 14, 26, 32, 34, 40, 48, 57, 59, 69, 73, 75, 77, 81, 82, 'ttat', 19, 44, 52, 81, 82, 'agaa', 6, 13, 15, 18, 27, 60, 82, 'ttatcaaatt', 44, 81, 82, 'aaaa', 15, 17, 18, 27, 48, 49, 82]


In [ ]:
from tinydb import TinyDB, Query
import re
import json
import re

In [17]:
reference = 'cactaagcacacagagaataatgtctagaatctgagtgccatgttatcaaattgtactgagactcttgcagtcacacaggct'
pattern_list = ['acac','cag', 'gtctagaat', '']
pattern_list_2 = ['*BIOTIN*-ACAC', 'TGTC', 'aaa', 'act gag act ctt gc']
pattern_length_list = []

In [19]:
for pattern in pattern_list:
    pattern_length = len(pattern)
    if pattern_length not in pattern_length_list:
        pattern_length_list.append(len(pattern))
        for i, nucleotide in enumerate(reference):
            ref_chunk = reference[i:i+pattern_length]
            db.insert({'sequence': ref_chunk, 'index':i+1})
            #+1 because of 0 indexing. This will give teh actual start site of the pattern.
    sequence_match = Query()
    print(db.search(sequence_match.sequence==pattern))

[{'index': 9, 'sequence': 'acac'}, {'index': 74, 'sequence': 'acac'}]
[{'index': 12, 'sequence': 'cag'}, {'index': 69, 'sequence': 'cag'}, {'index': 77, 'sequence': 'cag'}]


In [55]:
def pattern_find_tinydb(pattern_list, reference):
    matching = ''
    #matching_dict = {}
    for pattern in pattern_list:
        pattern_length = len(pattern)
        if pattern_length not in pattern_length_list:
            pattern_length_list.append(len(pattern))
            for i, nucleotide in enumerate(reference):
                ref_chunk = reference[i:i+pattern_length]
                db.insert({'sequence': ref_chunk, 'index':i+1})
                #+1 because of 0 indexing. This will give teh actual start site of the pattern.
        sequence_match = Query()
        matches = db.search(sequence_match.sequence==pattern)
        matches = json.dumps(matches)
        #print(matches)
        locations = re.findall(r'\d+', matches)
        #print(type(locations))
        #print([int(loc) for loc in matches.split(" ") if loc.isdigit()])
        #print("{} is found at {}".format(pattern, locations))
        #matching += str((db.search(sequence_match.sequence==pattern)))
        #matching_dict.add(db.search(sequence_match.sequence==pattern))
    return(matching)
print(pattern_find_tinydb(pattern_list, reference))

NameError: name 'pattern_length_list' is not defined

In [23]:
def pattern_find_in(pattern_list, refernece):
    pattern_match = []
    for pattern in pattern_list:
        if pattern in reference:
            pattern_match.append(pattern)
    return(pattern_match)

In [24]:
from timeit import timeit

In [25]:
print(pattern_find_tinydb(pattern_list, reference))
print ("pattern_find_tinydb:{}".format(timeit(
                                    "pattern_find_tinydb(pattern_list, reference)",
                                    "from __main__ import pattern_find_tinydb;"
                                    "reference='cactaagcacacagagaataatgtctagaatctgagtgccatgttatcaaattgtactgagactcttgcagtcacacaggct';" 
                                    "pattern_list=['acac','cag'] "
                                    , number=100000)))

print(pattern_find_in(pattern_list, reference))
print ("pattern_find_in:{}".format(timeit(
                                    "pattern_find_in(pattern_list, reference)",
                                    "from __main__ import pattern_find_in;"
                                    "reference='cactaagcacacagagaataatgtctagaatctgagtgccatgttatcaaattgtactgagactcttgcagtcacacaggct';" 
                                    "pattern_list=['acac','cag'] "
                                    , number=100000)))

print(approximate_patterns(reference, pattern_list, 0))
print ("approximate_patterns:{}".format(timeit(
                                    "approximate_patterns(reference, pattern_list, 0)",
                                    "from __main__ import approximate_patterns;"
                                    "reference='cactaagcacacagagaataatgtctagaatctgagtgccatgttatcaaattgtactgagactcttgcagtcacacaggct';" 
                                    "pattern_list=['acac','cag'] "
                                    , number=100000)))

In [34]:
print(pattern_length_list)
print(db.all())

[4]
[{'index': 1, 'sequence': 'cact'}, {'index': 2, 'sequence': 'acta'}, {'index': 3, 'sequence': 'ctaa'}, {'index': 4, 'sequence': 'taag'}, {'index': 5, 'sequence': 'aagc'}, {'index': 6, 'sequence': 'agca'}, {'index': 7, 'sequence': 'gcac'}, {'index': 8, 'sequence': 'caca'}, {'index': 9, 'sequence': 'acac'}, {'index': 10, 'sequence': 'caca'}, {'index': 11, 'sequence': 'acag'}, {'index': 12, 'sequence': 'caga'}, {'index': 13, 'sequence': 'agag'}, {'index': 14, 'sequence': 'gaga'}, {'index': 15, 'sequence': 'agaa'}, {'index': 16, 'sequence': 'gaat'}, {'index': 17, 'sequence': 'aata'}, {'index': 18, 'sequence': 'ataa'}, {'index': 19, 'sequence': 'taat'}, {'index': 20, 'sequence': 'aatg'}, {'index': 21, 'sequence': 'atgt'}, {'index': 22, 'sequence': 'tgtc'}, {'index': 23, 'sequence': 'gtct'}, {'index': 24, 'sequence': 'tcta'}, {'index': 25, 'sequence': 'ctag'}, {'index': 26, 'sequence': 'taga'}, {'index': 27, 'sequence': 'agaa'}, {'index': 28, 'sequence': 'gaat'}, {'index': 29, 'sequence'

In [15]:
db.purge()
pattern_length_list = []
print(db.all())

[]


In [41]:
# def reference_dict(pattern_length, reference):
#     for i, nucleotide in enumerate(reference):
#         query = reference[i:i+pattern_length]
#         query_number = pattern_to_number(query)
#         if len(query) == pattern_length and query not in query_dict:
#             query_dict[query]=str(i+1)+','
#             #+1 because of 0 indexing. This will give teh actual start site of the pattern.
#         elif len(query) == pattern_length:
#             query_dict[query]+=str(i+1)+','
            
#     db.insert(query_dict)
#     return (db.all())

In [42]:
def symbol_to_number(symbol):
    dict_symbol = {'A':0, 'C':1, 'G':2, 'T':3}
    return dict_symbol[symbol]

def pattern_to_number(pattern):
    pattern = pattern.upper().replace(" ","")
    regex = re.compile('[^agctuAGCTU]')
    pattern = regex.sub('', pattern)
    if not pattern:
        return 0
    symbol = pattern[-1]
    prefix = pattern[0:-1]
    return 4*pattern_to_number(prefix) + symbol_to_number(symbol)